# Análisis de índices bursátiles

### Importación de librerías

In [182]:
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import os

### Importar desde Yahoo Finance los CSVs
Una vez importado,  comento todo el codigo para que al probarlo no genere duplicadaos

In [183]:
"""
# Diccionario con los índices y sus tickers de Yahoo Finance
indices = {
    'S&P 500': '^GSPC',
    'Nasdaq 100': '^NDX',
    'Dow Jones Industrial Average': '^DJI',
    'Russell 2000': '^RUT',
    'DAX 40': '^GDAXI',
    'CAC 40': '^FCHI',
    'FTSE 100': '^FTSE',
    'Nikkei 225': '^N225',
    'Hang Seng Index': '^HSI',
    'Euro Stoxx 50': '^STOXX50E'
}

# Rango de fechas
start_date = '2018-01-01'
end_date = '2024-12-31'

# Descargar y guardar los datos en la carpeta ../data/raw
for nombre, ticker in indices.items():
    data = yf.download(ticker, start=start_date, end=end_date)
    file_path = f'../data/raw/{nombre.replace(" ", "_").replace("/", "_")}.csv'
    data.to_csv(file_path)
    print(f'Datos guardados para {nombre} en {file_path}')
"""

'\n# Diccionario con los índices y sus tickers de Yahoo Finance\nindices = {\n    \'S&P 500\': \'^GSPC\',\n    \'Nasdaq 100\': \'^NDX\',\n    \'Dow Jones Industrial Average\': \'^DJI\',\n    \'Russell 2000\': \'^RUT\',\n    \'DAX 40\': \'^GDAXI\',\n    \'CAC 40\': \'^FCHI\',\n    \'FTSE 100\': \'^FTSE\',\n    \'Nikkei 225\': \'^N225\',\n    \'Hang Seng Index\': \'^HSI\',\n    \'Euro Stoxx 50\': \'^STOXX50E\'\n}\n\n# Rango de fechas\nstart_date = \'2018-01-01\'\nend_date = \'2024-12-31\'\n\n# Descargar y guardar los datos en la carpeta ../data/raw\nfor nombre, ticker in indices.items():\n    data = yf.download(ticker, start=start_date, end=end_date)\n    file_path = f\'../data/raw/{nombre.replace(" ", "_").replace("/", "_")}.csv\'\n    data.to_csv(file_path)\n    print(f\'Datos guardados para {nombre} en {file_path}\')\n'

### Carga de datos raw

In [184]:
df_sp500 = pd.read_csv("../data/raw/sp_500.csv").iloc[2:]
df_cac40 = pd.read_csv("../data/raw/cac_40.csv").iloc[2:]
df_dax40 = pd.read_csv("../data/raw/dax_40.csv").iloc[2:]
df_dowjones = pd.read_csv("../data/raw/dow_jones.csv").iloc[2:]
df_eurostoxx50 = pd.read_csv("../data/raw/euro_stoxx_50.csv").iloc[2:]
df_ftse100 = pd.read_csv("../data/raw/ftse_100.csv").iloc[2:]
df_hangseng = pd.read_csv("../data/raw/hang_seng.csv").iloc[2:]
df_nasdaq100 = pd.read_csv("../data/raw/nasdaq_100.csv").iloc[2:]
df_nikkei225 = pd.read_csv("../data/raw/nikkei_225.csv").iloc[2:]
df_russell2000 = pd.read_csv("../data/raw/russell_2000.csv").iloc[2:]

📈 Función de análisis financiero por índice:  

Esta función aplica un conjunto de cálculos estándar de análisis financiero a cualquier índice bursátil. Dado un DataFrame con precios diarios (Open, High, Low, Close, Volume), añade columnas clave para evaluar su rendimiento y comportamiento a lo largo del tiempo.

🔧 Cálculos incluidos:  
- Rentabilidad diaria y porcentual
- Medias móviles de 10 y 50 días
- Rentabilidad acumulada (desde el primer día)
- Caída máxima (drawdown) respecto al máximo histórico
- Volatilidad móvil de 30 días
- Ratio de Sharpe móvil de 30 días

📊 Indicadores agregados que se imprimen:
- CAGR (Tasa de crecimiento anual compuesta)
- Caída máxima promedio
- Volatilidad media
- Sharpe medio

In [185]:
df_sp500.rename(columns={"Price": "Date"}, inplace=True)
df_cac40.rename(columns={"Price": "Date"}, inplace=True)
df_dax40.rename(columns={"Price": "Date"}, inplace=True)
df_dowjones.rename(columns={"Price": "Date"}, inplace=True)
df_eurostoxx50.rename(columns={"Price": "Date"}, inplace=True)
df_ftse100.rename(columns={"Price": "Date"}, inplace=True)
df_hangseng.rename(columns={"Price": "Date"}, inplace=True)
df_nasdaq100.rename(columns={"Price": "Date"}, inplace=True)
df_nikkei225.rename(columns={"Price": "Date"}, inplace=True)
df_russell2000.rename(columns={"Price": "Date"}, inplace=True)

In [186]:
df_sp500.columns = df_sp500.columns.str.lower()
df_cac40.columns = df_cac40.columns.str.lower()
df_dax40.columns = df_dax40.columns.str.lower()
df_dowjones.columns = df_dowjones.columns.str.lower()
df_eurostoxx50.columns = df_eurostoxx50.columns.str.lower()
df_ftse100.columns = df_ftse100.columns.str.lower()
df_hangseng.columns = df_hangseng.columns.str.lower()
df_nasdaq100.columns = df_nasdaq100.columns.str.lower()
df_nikkei225.columns = df_nikkei225.columns.str.lower()
df_russell2000.columns = df_russell2000.columns.str.lower()


In [187]:
df_sp500 = df_sp500.set_index("date")
df_cac40 = df_cac40.set_index("date")
df_dax40 = df_dax40.set_index("date")
df_dowjones = df_dowjones.set_index("date")
df_eurostoxx50 = df_eurostoxx50.set_index("date")
df_ftse100 = df_ftse100.set_index("date")
df_hangseng = df_hangseng.set_index("date")
df_nasdaq100 = df_nasdaq100.set_index("date")
df_nikkei225 = df_nikkei225.set_index("date")
df_russell2000 = df_russell2000.set_index("date")


### Procesamiento y Limpieza de datos

In [188]:
def procesar_indice(df):
    # Calcula la rentabilidad diaria como cambio porcentual
    df['rentabilidad'] = df['close'].pct_change()
    df['rentabilidad_%'] = df['rentabilidad'] * 100  # Rentabilidad en porcentaje

    # Medias móviles para detectar tendencias a corto y medio plazo
    df['media_movil_10d'] = df['close'].rolling(window=10).mean()
    df['media_movil_50d'] = df['close'].rolling(window=50).mean()

    # Rentabilidad acumulada para evaluar la evolución total
    df['rentabilidad_acumulada'] = (1 + df['rentabilidad']).cumprod()

    # Máximo acumulado para calcular caídas desde el pico
    df['maximo_acumulado'] = df['rentabilidad_acumulada'].cummax()

    # Caída máxima (drawdown) relativa al máximo acumulado
    df['caida_maxima'] = df['rentabilidad_acumulada'] / df['maximo_acumulado'] - 1

    # Se elimina columna auxiliar para limpieza
    df = df.drop(columns=['maximo_acumulado'])

    # Volatilidad a 30 días como desviación estándar de rentabilidades
    df['volatilidad_30d'] = df['rentabilidad'].rolling(window=30).std()

    # Ratio de Sharpe simple a 30 días: rentabilidad media / volatilidad
    df['sharpe_30d'] = df['rentabilidad'].rolling(window=30).mean() / df['volatilidad_30d']

    # Asegura que el índice sea datetime para cálculos temporales
    df.index = pd.to_datetime(df.index)

    # Cálculo de años totales del período analizado
    años = (df.index[-1] - df.index[0]).days / 365

    # Precios inicial y final para rendimiento total y CAGR
    precio_inicial = df['close'].iloc[0]
    precio_final = df['close'].iloc[-1]

    # Rendimiento total en porcentaje entre inicio y fin del período
    rendimiento_18_24 = (precio_final / precio_inicial - 1) * 100

    # Indicadores resumen para análisis general
    indicadores = {
        'CAGR': (precio_final / precio_inicial) ** (1 / años) - 1,  # Crecimiento anual compuesto
        'drawdown_promedio': df['caida_maxima'].mean(),
        'volatilidad_media': df['volatilidad_30d'].mean(),
        'sharpe_medio': df['sharpe_30d'].mean(),
        'rendimiento_18_24': rendimiento_18_24,
        'volumen_medio': df['volume'].astype(float).mean()
    }
    return df, indicadores


In [189]:
for df in [df_sp500, df_cac40, df_dax40, df_dowjones, df_eurostoxx50, df_ftse100, df_hangseng, df_nasdaq100, df_nikkei225, df_russell2000]:
    # Convierte 'close' a numérico y elimina filas con valores no válidos
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df.dropna(subset=['close'], inplace=True)
    
    # Procesa el DataFrame con la función definida antes
    df_proc, _ = procesar_indice(df)
    
    # La función devuelve copia, así que actualizamos el df original con las nuevas columnas
    for col in ['rentabilidad', 'rentabilidad_%', 'media_movil_10d', 'media_movil_50d',
                'rentabilidad_acumulada', 'caida_maxima', 'volatilidad_30d', 'sharpe_30d']:
        df[col] = df_proc[col]




In [190]:
# Guardar cada DataFrame procesado en un archivo CSV separado
df_sp500.to_csv('../data/processed/sp500_procesado.csv')
df_cac40.to_csv('../data/processed/cac40_procesado.csv')
df_dax40.to_csv('../data/processed/dax40_procesado.csv')
df_dowjones.to_csv('../data/processed/dowjones_procesado.csv')
df_eurostoxx50.to_csv('../data/processed/eurostoxx50_procesado.csv')
df_ftse100.to_csv('../data/processed/ftse100_procesado.csv')
df_hangseng.to_csv('../data/processed/hangseng_procesado.csv')
df_nasdaq100.to_csv('../data/processed/nasdaq100_procesado.csv')
df_nikkei225.to_csv('../data/processed/nikkei225_procesado.csv')
df_russell2000.to_csv('../data/processed/russell2000_procesado.csv')

In [191]:
# Diccionario con nombres y DataFrames de los índices
dfs = {
    "🇺🇸 SP 500": df_sp500,
    "🇫🇷 CAC 40": df_cac40,
    "🇩🇪 DAX 40": df_dax40,
    "🇺🇸 Dow Jones": df_dowjones,
    "🇪🇺 Euro Stoxx 50": df_eurostoxx50,
    "🇬🇧 FTSE 100": df_ftse100,
    "🇭🇰 Hang Seng": df_hangseng,
    "🇺🇸 Nasdaq 100": df_nasdaq100,
    "🇯🇵 Nikkei 225": df_nikkei225,
    "🇺🇸 Russell 2000": df_russell2000,
}

resultados = {}

# Procesa cada DataFrame para obtener sus indicadores y guardarlos en un dict
for nombre, df in dfs.items():
    _, indicadores = procesar_indice(df)
    resultados[nombre] = indicadores

# Convierte el diccionario de indicadores en un DataFrame
df_allindex = pd.DataFrame.from_dict(resultados, orient='index')
df_allindex = df_allindex.reset_index()  # Pone los índices como columna
df_allindex.rename(columns={'index': 'indice'}, inplace=True)  # Renombra columna
df_allindex = df_allindex.set_index("indice")  # Usa la columna 'indice' como índice

# Guarda el resumen de todos los índices en CSV
df_allindex.to_csv('../data/processed/allindex_procesado.csv')


In [ ]:
def graficar_precio_cierre(df, nombre_indice):
    # Asegura que el índice sea tipo fecha
    df.index = pd.to_datetime(df.index)

    # Crear figura con Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['close'],
        mode='lines',
        line=dict(color='deepskyblue', width=4),
        name=nombre_indice
    ))

    # Configuración del diseño y estilo del gráfico
    fig.update_layout(
        title=dict(
            text=nombre_indice,
            x=0.5,
            xanchor='center',
            font=dict(size=70)
        ),
        xaxis_title='Años',
        yaxis_title='Precio ($)',
        xaxis=dict(tickfont=dict(size=45)),
        yaxis=dict(tickfont=dict(size=45)),
        template='plotly_dark',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white', size=45),
        width=3840,
        height=2160,
        margin=dict(t=150, l=80, r=80, b=80)  # Más espacio arriba para el título
    )

    # Crear carpeta si no existe para guardar las imágenes
    carpeta = "../plots"
    os.makedirs(carpeta, exist_ok=True)

    # Limpiar el nombre para que no tenga emojis ni caracteres problemáticos en el archivo
    nombre_limpio = nombre_indice.encode('ascii', 'ignore').decode().replace(" ", "").replace("-", "").replace("&", "").replace("́", "")
    ruta_salida = os.path.abspath(os.path.join(carpeta, f"Grafica_{nombre_limpio}.png"))

    print("Guardando gráfico en:", ruta_salida)

    # Guardar la imagen, con manejo de errores
    try:
        pio.write_image(fig, ruta_salida, width=3840, height=2160, scale=3)
        print(f"✅ Gráfico guardado correctamente en: {ruta_salida}")
    except Exception as e:
        print("❌ Error al guardar la imagen:", e)


# Llamadas para graficar cada índice
graficar_precio_cierre(df_sp500, '🇺🇸 S&P 500')
graficar_precio_cierre(df_cac40, '🇫🇷 CAC 40')
graficar_precio_cierre(df_dax40, '🇩🇪 DAX 40')
graficar_precio_cierre(df_dowjones, '🇺🇸 Dow Jones')
graficar_precio_cierre(df_eurostoxx50, '🇪🇺 Euro Stoxx 50')
graficar_precio_cierre(df_ftse100, '🇬🇧 FTSE 100')
graficar_precio_cierre(df_hangseng, '🇭🇰 Hang Seng')
graficar_precio_cierre(df_nasdaq100, '🇺🇸 Nasdaq 100')
graficar_precio_cierre(df_nikkei225, '🇯🇵 Nikkei 225')
graficar_precio_cierre(df_russell2000, '🇺🇸 Russell 2000')


Guardando gráfico en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_SP500.png
✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_SP500.png
Guardando gráfico en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_CAC40.png
✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_CAC40.png
Guardando gráfico en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_DAX40.png
✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_DAX40.png
Guardando gráfico en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_DowJones.png
✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/p

In [193]:
# Definir una escala de colores personalizada en tonos azules
custom_blue_scale = [
    [0, "#A9D6E5"],  # Azul claro
    [0.5, "#1976D2"], # Azul medio
    [1, "#0D3B66"]   # Azul oscuro
]

# Ordenar el DataFrame por rendimiento para que el azul oscuro quede arriba en el gráfico horizontal
df_sorted = df_allindex.reset_index().sort_values(by='rendimiento_18_24', ascending=True)

# Crear gráfico de barras horizontales con color según rendimiento
fig_bar = px.bar(
    df_sorted,
    y='indice',
    x='rendimiento_18_24',
    color='rendimiento_18_24',
    color_continuous_scale=custom_blue_scale,
    orientation='h',
    title='Rendimiento 2018 - 2024',
    labels={'indice': 'Índice Bursátil', 'rendimiento_18_24': 'Rendimiento(%)'},
)

# Personalizar barras: borde negro y texto con porcentaje al final de cada barra
fig_bar.update_traces(
    marker_line_color='black',
    marker_line_width=1.5,
    text=df_sorted['rendimiento_18_24'].apply(lambda x: f"{x:.0f}%"),
    textposition='outside',
    textfont=dict(color='white', size=50),  # Tamaño grande para pantallas 4K
)

# Ajustes visuales generales y estilo oscuro
fig_bar.update_layout(
    template='plotly_dark',
    width=3840,
    height=2160,
    title=dict(
        text='<b>Rendimiento</b>',
        x=0.5,
        xanchor='center',
        font=dict(size=70)  # Título grande para alta resolución
    ),
    xaxis_title=None,
    yaxis_title=None,
    plot_bgcolor='black',
    paper_bgcolor='black',
    xaxis=dict(
        showgrid=False,
        showticklabels=False,
        tickfont=dict(size=45)  # Por si se activan las etiquetas
    ),
    yaxis=dict(
        showgrid=False,
        tickfont=dict(size=45)  # Etiquetas eje Y grandes y legibles
    ),
    coloraxis_showscale=False  # Ocultar la barra de escala de colores
)

#fig_bar.show()

# Guardar gráfico en alta calidad 4K
pio.write_image(fig_bar, '../plots/grafica_rendimiento.png', width=3840, height=2160, scale=3)


In [194]:
# Ordenar el DataFrame
df_sorted = df_allindex.reset_index().sort_values(by='CAGR', ascending=True)

# Crear la gráfica de barras
fig_bar = px.bar(
    df_sorted,
    x='indice',
    y='CAGR',
    color='CAGR',
    color_continuous_scale='Blues',
    labels={'indice': '', 'CAGR': 'Tasa de Crecimiento Anual (%)'},
)

# Personalizar trazas
fig_bar.update_traces(
    marker_line_color='black',
    marker_line_width=2,
    text=df_sorted['CAGR'].apply(lambda x: f"{x:.0%}"),
    textposition='outside',
    textfont=dict(color='white', size=36)  # Tamaño del texto
)

# Ajustar diseño para 4K
fig_bar.update_layout(
    template='plotly_dark',
    xaxis_tickangle=-45,
    yaxis_tickformat=".0%",
    coloraxis_showscale=False,
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis_showticklabels=False,
    yaxis_title='',
    xaxis=dict(tickfont=dict(size=38)),  # Eje X
    font=dict(color='white', size=42),   # Fuente general
    height=2160,
    width=3840,
    margin=dict(t=150, b=150),
    title=dict(
        text='<b>Crecimiento Anual Promedio</b>',
        x=0.5,
        xanchor='center',
        font=dict(size=72)
    ),
)

# Mostrar en pantalla
#fig_bar.show()

# Guardar como imagen
carpeta = "../plots"
os.makedirs(carpeta, exist_ok=True)
ruta_salida = os.path.abspath(os.path.join(carpeta, "Grafica_Crec_Anual.png"))

print("Guardando gráfico en:", ruta_salida)

try:
    pio.write_image(fig_bar, ruta_salida, width=3840, height=2160, scale=3)
    print(f"✅ Gráfico guardado correctamente en: {ruta_salida}")
except Exception as e:
    print("❌ Error al guardar la imagen:", e)



Guardando gráfico en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_Crec_Anual.png
✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_Crec_Anual.png


In [203]:
import os
import plotly.express as px
import plotly.io as pio

# Crear el gráfico Treemap sin mostrar el root gris
fig = px.treemap(
    df_allindex.reset_index(),
    path=[px.Constant(""), 'indice'],  # Evita el bloque root gris
    values='volumen_medio',
    color='volumen_medio',
    color_continuous_scale='Blues',
    title='Volumen Medio de Transacciones/Día'
)

# Fondo y estilo para pantalla grande
fig.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    width=3840,
    height=2160,
    font=dict(color='white', size=40),
    title=dict(
        text='<b>Volumen Medio de Transacciones/Día</b>',
        x=0.5,
        xanchor='center',
        font=dict(size=70)
    ),
    margin=dict(t=150, l=50, r=50, b=50)
)
#fig.show()
# Guardar imagen en alta resolución
carpeta = "../plots"
os.makedirs(carpeta, exist_ok=True)
ruta_salida = os.path.abspath(os.path.join(carpeta, "Grafica_Volumen.png"))

pio.write_image(fig, ruta_salida, width=3840, height=2160, scale=3)
print(f"✅ Gráfico guardado correctamente en: {ruta_salida}")


✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_Volumen.png


In [196]:
# Ordenar índices por caída promedio de mayor a menor
df_sorted = df_allindex.reset_index().sort_values(by='drawdown_promedio', ascending=False)

# Crear gráfico de barras verticales con color según caída promedio
fig = px.bar(
    df_sorted,
    x='indice',
    y='drawdown_promedio',
    color='drawdown_promedio',
    color_continuous_scale=custom_blue_scale,
    labels={'drawdown_promedio': '% Drawdown Promedio', 'indice': 'Índice'},
)

# Añadir texto con porcentaje fuera de las barras y borde negro
fig.update_traces(
    text=df_sorted['drawdown_promedio'].apply(lambda x: f"{x:.2f}%"),
    textposition='outside',
    marker_line_color='black',
    marker_line_width=1.5,
    textfont=dict(size=36, color='white')
)

# Configurar diseño visual, estilo oscuro y ajustes para pantallas 4K
fig.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    width=3840,
    height=2160,
    font=dict(color='white', size=45),
    xaxis=dict(
        showgrid=False,
        tickangle=-45,
        tickfont=dict(size=40),
        title=None
    ),
    yaxis=dict(
        showgrid=False,
        showticklabels=False,
        title_text=''
    ),
    title=dict(
        text='<b>Caída Promedio por Índice</b>',
        x=0.5,
        xanchor='center',
        font=dict(size=70)
    ),
    margin=dict(t=150, b=100),
    coloraxis_showscale=False  # Ocultar barra de colores
)

#fig.show()

# Crear carpeta si no existe y definir ruta para guardar imagen
carpeta = "../plots"
os.makedirs(carpeta, exist_ok=True)
ruta_salida = os.path.abspath(os.path.join(carpeta, "Grafica_DrawdownPromedio.png"))

print("Guardando gráfico en:", ruta_salida)
try:
    pio.write_image(fig, ruta_salida, width=3840, height=2160, scale=3)  # Guardar en alta resolución
    print(f"✅ Gráfico guardado correctamente en: {ruta_salida}")
except Exception as e:
    print("❌ Error al guardar la imagen:", e)


Guardando gráfico en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_DrawdownPromedio.png
✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_DrawdownPromedio.png


In [212]:
import os
import plotly.express as px
import plotly.io as pio

# Escala de azules personalizada para el color de las barras
custom_blue_scale = [
    [0, "#A9D6E5"], 
    [0.5, "#1976D2"], 
    [1, "#0D3B66"]
]

# Ordenar DataFrame por Sharpe medio ascendente para el gráfico horizontal
df_sorted = df_allindex.reset_index().sort_values(by='sharpe_medio', ascending=True)

# Crear gráfico de barras horizontal con color según valor de Sharpe medio
fig = px.bar(
    df_sorted,
    y='indice',
    x='sharpe_medio',
    orientation='h',
    color='sharpe_medio',
    color_continuous_scale=custom_blue_scale,
    labels={'sharpe_medio': 'Ratio de Sharpe Medio', 'indice': 'Índice'}
)

# Añadir etiquetas con formato numérico (2 decimales), sin símbolo %
fig.update_traces(
    text=df_sorted['sharpe_medio'].apply(lambda x: f"{x:.2f}"),
    textposition='outside',
    marker_line_color='black',
    marker_line_width=1.5,
    textfont=dict(size=36, color='white')
)

# Ajustar layout para pantalla grande, estilo oscuro y espacio Y mínimo
fig.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    width=3840,
    height=2160,
    font=dict(color='white', size=45),
    title=dict(
        text='<b>Ratio Riesgo/Beneficio</b>',
        x=0.5,
        xanchor='center',
        font=dict(size=70)
    ),
    xaxis=dict(
        showgrid=False,
        showticklabels=True,
        title=None,
        range=[df_sorted['sharpe_medio'].min() - 0.1, df_sorted['sharpe_medio'].max() + 0.1],
        tickfont=dict(size=40)
    ),
    yaxis=dict(
        showgrid=False,
        title=None,
        tickfont=dict(size=40)
    ),
    coloraxis_showscale=False,
    margin=dict(t=150, l=5, r=100, b=100)  # margen izquierdo mínimo
)

# Crear carpeta si no existe y definir ruta para guardar el gráfico en HD
carpeta = "../plots"
os.makedirs(carpeta, exist_ok=True)
ruta_salida = os.path.abspath(os.path.join(carpeta, "Grafica_SharpeMedio.png"))

print("Guardando gráfico en:", ruta_salida)
try:
    pio.write_image(fig, ruta_salida, width=3840, height=2160, scale=3)
    print(f"✅ Gráfico guardado correctamente en: {ruta_salida}")
except Exception as e:
    print("❌ Error al guardar la imagen:", e)


Guardando gráfico en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_SharpeMedio.png
✅ Gráfico guardado correctamente en: /Users/omarayarizarrougui/Desktop/BOOTCAMP_DATA_SCIENCE/stock_indices_eda/plots/Grafica_SharpeMedio.png
